<font size='4'><b>Notebook content:</b> Scapes Tokyo's wards data from Wikipedia and wrangles them into a suitable format. Utilises Foursquare API for collecting data about Indian restaurants in Tokyo. Uses Folium for map representation. Wards will be ordered into 3 distinct groups based on number of Indian restaurants.</font>

<br/><br/>

In [1]:
# Importing libraries.
import numpy as np
import pandas as pd 
from pandas.io.json import json_normalize

import json
import requests 
from bs4 import BeautifulSoup 

# Geo-spatial libraries.
from geopy.geocoders import Nominatim
import folium

import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
# Ignoring warnings.
import warnings
warnings.filterwarnings('ignore')

In [3]:
response= requests.get('https://en.wikipedia.org/wiki/Special_wards_of_Tokyo').text
soup = BeautifulSoup(response,'lxml')

In [4]:
Wards= soup.find('table', {'class':'wikitable sortable'})

In [5]:
Name=[]
Kanji = []
Pop = []
Density = []
num = []
flag = []
Area = []
District = []

for row in Wards.findAll("tr"):   
    Ward = row.findAll('td')
    print (Ward)
    # Extracting the body leaving heading
    if len(Ward)==8: 
        print (Ward[0])
        num.append(Ward[0].find(text=True))
        flag.append(Ward[1].findAll('a')) 
        Name.append(Ward[2])
        Kanji.append(Ward[3].find(text=True))
        Pop.append(Ward[4])
        Area.append(Ward[5].find(text=True))
        District.append(Ward[7].find(text=True))
            

[]
[<td>01</td>, <td><a class="image" href="/wiki/File:Flag_of_Chiyoda,_Tokyo.svg"><img alt="Flag of Chiyoda, Tokyo.svg" class="thumbborder" data-file-height="540" data-file-width="810" decoding="async" height="33" src="//upload.wikimedia.org/wikipedia/commons/thumb/e/ec/Flag_of_Chiyoda%2C_Tokyo.svg/50px-Flag_of_Chiyoda%2C_Tokyo.svg.png" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/e/ec/Flag_of_Chiyoda%2C_Tokyo.svg/75px-Flag_of_Chiyoda%2C_Tokyo.svg.png 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/e/ec/Flag_of_Chiyoda%2C_Tokyo.svg/100px-Flag_of_Chiyoda%2C_Tokyo.svg.png 2x" width="50"/></a></td>, <td><a href="/wiki/Chiyoda,_Tokyo" title="Chiyoda, Tokyo">Chiyoda</a></td>, <td>千代田区
</td>, <td><span style="visibility:hidden;color:transparent;">0</span><span style="visibility:hidden;color:transparent;">0</span>59,441</td>, <td><span style="visibility:hidden;color:transparent;">0</span>5,100</td>, <td><span style="visibility:hidden;color:transparent;">0</span>11.66
</td>, <t

In [6]:
Area[3]

'18,620'

In [7]:
# Replacing with the true value 5100.
Area = ['5100' if x=='0' else x for x in Area]
Area_new = []

In [8]:
# Changing the type of Area list. 
for a in range(len(Area)):
    area=Area[a].replace(",","")
    print (area)
    Area_new.append(area)

Area_new=[int(a) for a in Area_new]

5100
14460
12180
18620
19790
19830
18910
12510
17180
19110
11910
15690
15080
21350
16750
22650
16740
21030
17670
15120
12660
12850
13750


In [9]:
new_names = []
for n in range(len(Name)):
    print (Name[n])
    names = Name[n].findAll('a')
    new_names.append(names) 

print (new_names)

<td><a href="/wiki/Chiyoda,_Tokyo" title="Chiyoda, Tokyo">Chiyoda</a></td>
<td><a href="/wiki/Ch%C5%AB%C5%8D,_Tokyo" title="Chūō, Tokyo">Chūō</a></td>
<td><a href="/wiki/Minato,_Tokyo" title="Minato, Tokyo">Minato</a></td>
<td><a href="/wiki/Shinjuku" title="Shinjuku">Shinjuku</a></td>
<td><a href="/wiki/Bunky%C5%8D" title="Bunkyō">Bunkyō</a></td>
<td><a href="/wiki/Tait%C5%8D" title="Taitō">Taitō</a></td>
<td><a href="/wiki/Sumida,_Tokyo" title="Sumida, Tokyo">Sumida</a></td>
<td><a class="mw-redirect" href="/wiki/Koto,_Tokyo" title="Koto, Tokyo">Kōtō</a></td>
<td><a href="/wiki/Shinagawa" title="Shinagawa">Shinagawa</a></td>
<td><a href="/wiki/Meguro" title="Meguro">Meguro</a></td>
<td><span style="display:none">Ota</span><a href="/wiki/%C5%8Cta,_Tokyo" title="Ōta, Tokyo">Ōta</a></td>
<td><a href="/wiki/Setagaya" title="Setagaya">Setagaya</a></td>
<td><a href="/wiki/Shibuya" title="Shibuya">Shibuya</a></td>
<td><a href="/wiki/Nakano,_Tokyo" title="Nakano, Tokyo">Nakano</a></td>
<td><

In [10]:
flat_new_names = [item for sublist in new_names for item in sublist]
print (flat_new_names)

[<a href="/wiki/Chiyoda,_Tokyo" title="Chiyoda, Tokyo">Chiyoda</a>, <a href="/wiki/Ch%C5%AB%C5%8D,_Tokyo" title="Chūō, Tokyo">Chūō</a>, <a href="/wiki/Minato,_Tokyo" title="Minato, Tokyo">Minato</a>, <a href="/wiki/Shinjuku" title="Shinjuku">Shinjuku</a>, <a href="/wiki/Bunky%C5%8D" title="Bunkyō">Bunkyō</a>, <a href="/wiki/Tait%C5%8D" title="Taitō">Taitō</a>, <a href="/wiki/Sumida,_Tokyo" title="Sumida, Tokyo">Sumida</a>, <a class="mw-redirect" href="/wiki/Koto,_Tokyo" title="Koto, Tokyo">Kōtō</a>, <a href="/wiki/Shinagawa" title="Shinagawa">Shinagawa</a>, <a href="/wiki/Meguro" title="Meguro">Meguro</a>, <a href="/wiki/%C5%8Cta,_Tokyo" title="Ōta, Tokyo">Ōta</a>, <a href="/wiki/Setagaya" title="Setagaya">Setagaya</a>, <a href="/wiki/Shibuya" title="Shibuya">Shibuya</a>, <a href="/wiki/Nakano,_Tokyo" title="Nakano, Tokyo">Nakano</a>, <a href="/wiki/Suginami" title="Suginami">Suginami</a>, <a href="/wiki/Toshima" title="Toshima">Toshima</a>, <a href="/wiki/Kita,_Tokyo" title="Kita, Tok

In [11]:
Wards_names= [] 
for name_wards in flat_new_names:
        Wards_names.append(name_wards.get('title'))

print (Wards_names)

['Chiyoda, Tokyo', 'Chūō, Tokyo', 'Minato, Tokyo', 'Shinjuku', 'Bunkyō', 'Taitō', 'Sumida, Tokyo', 'Koto, Tokyo', 'Shinagawa', 'Meguro', 'Ōta, Tokyo', 'Setagaya', 'Shibuya', 'Nakano, Tokyo', 'Suginami', 'Toshima', 'Kita, Tokyo', 'Arakawa, Tokyo', 'Itabashi', 'Nerima', 'Adachi, Tokyo', 'Katsushika', 'Edogawa, Tokyo']


In [12]:
# Removing 'Tokyo' from certain wards.
replacement={'Chiyoda, Tokyo':'Chiyoda', 'Chūō, Tokyo':'Chuo', 'Minato, Tokyo':'Minato', 
               'Sumida, Tokyo':'Sumida', 'Koto, Tokyo':'Koto', 'Ōta, Tokyo':'Ota', 'Nakano, Tokyo':'Nakano', 
               'Kita, Tokyo':'Kita', 'Arakawa, Tokyo':'Arakawa', 'Adachi, Tokyo':'Adachi', 'Edogawa, Tokyo':'Edogawa'}


Wards_names1 = [replacement.get(name,name) for name in Wards_names]
print(Wards_names1)

['Chiyoda', 'Chuo', 'Minato', 'Shinjuku', 'Bunkyō', 'Taitō', 'Sumida', 'Koto', 'Shinagawa', 'Meguro', 'Ota', 'Setagaya', 'Shibuya', 'Nakano', 'Suginami', 'Toshima', 'Kita', 'Arakawa', 'Itabashi', 'Nerima', 'Adachi', 'Katsushika', 'Edogawa']


In [13]:
population = []
for p in range(len(Pop)):
    print ((Pop[p]))
    pops = Pop[p].text[1:9]
    print (Pop[p].text[1:9])
    population.append(pops) 
print (population)

<td><span style="visibility:hidden;color:transparent;">0</span><span style="visibility:hidden;color:transparent;">0</span>59,441</td>
059,441
<td><span style="visibility:hidden;color:transparent;">0</span>147,620</td>
147,620
<td><span style="visibility:hidden;color:transparent;">0</span>248,071</td>
248,071
<td><span style="visibility:hidden;color:transparent;">0</span>339,211</td>
339,211
<td><span style="visibility:hidden;color:transparent;">0</span>223,389</td>
223,389
<td><span style="visibility:hidden;color:transparent;">0</span>200,486</td>
200,486
<td><span style="visibility:hidden;color:transparent;">0</span>260,358</td>
260,358
<td><span style="visibility:hidden;color:transparent;">0</span>502,579</td>
502,579
<td><span style="visibility:hidden;color:transparent;">0</span>392,492</td>
392,492
<td><span style="visibility:hidden;color:transparent;">0</span>280,283</td>
280,283
<td><span style="visibility:hidden;color:transparent;">0</span>722,608</td>
722,608
<td><span style="v

In [15]:
population_new = []
for po in range(len(population)):
    pop=population[po].replace(",","")
    print (pop)
    population_new.append(pop)

population_new=[int(s1) for s1 in population_new]

059441
147620
248071
339211
223389
200486
260358
502579
392492
280283
722608
910868
227850
332902
570483
294673
345063
213648
569225
726748
674067
447140
685899


In [16]:
districts_replace = {'Nagatachō':'Nagatacho', 'Hongō':'Hongo', 'Kinshichō':'Kinshicho', 'Ōmori': 'Omori', 
                     'Kōenji':'Koenji', 'Arakawa, Machiya, ':'Arakawa', 'Ayase, ':'Ayase', 'Kasai, Koiwa\n':'Kasai'}


District_names1 = [districts_replace.get(dis,dis) for dis in District]
print(District_names1)

['Nagatacho', 'Nihonbashi', 'Odaiba', 'Shinjuku', 'Hongo', 'Ueno', 'Kinshicho', 'Kiba', 'Shinagawa', 'Meguro', 'Omori', 'Shimokitazawa', 'Shibuya', 'Nakano', 'Koenji', 'Ikebukuro', 'Akabane', 'Arakawa', 'Itabashi', 'Nerima', 'Ayase', 'Tateishi', 'Kasai']


In [18]:
df=pd.DataFrame(Wards_names1,columns=['Ward'])
df['Area_SqKm'] = Area_new
df['Population'] = population_new
df['District'] = District_names1
# Starting the index from 0.
df.index = np.arange(1, len(df) + 1)

In [19]:
geolocator = Nominatim(user_agent="Tokyo_explorer_new")
df['Dist_Coord']= df['District'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))
df[['Latitude', 'Longitude']] = df['Dist_Coord'].apply(pd.Series)

df.drop(['Dist_Coord'], axis=1, inplace=True)

In [20]:
Lat = df['Latitude'].tolist()
Long= df['Longitude'].tolist()
print ("latitude list: ", Lat)
print ("Longitude list: ", Long)

latitude list:  [35.675618, 35.68406775, 35.626722, 35.6937632, 35.1753761, 35.7117896, 35.6963122, 23.0131338, 35.599252, 35.62125, 35.5884735, 35.6616779, 35.6645956, 35.718123, 35.7049419, 35.7310839, 35.7781394, 35.737529, 35.774143, 35.74836, 35.4460475, 33.4817913, -5.3498001]
Longitude list:  [139.7434685, 139.77450291683806, 139.7721007, 139.7036319, 137.0134764, 139.7775976, 139.8150431, -80.8328748, 139.73891, 139.688014, 139.7279334, 139.6663345, 139.6987107, 139.664468, 139.649909, 139.7089164, 139.7207999, 139.78131, 139.681209, 139.638735, 139.430823, 131.4781536, 21.424098]


In [22]:
# Replacing incorrect Coordinates.
lat_replace = {32.5093796:35.7088, 23.0131338:35.6722, -38.9047057:35.5884, -5.3498001:35.6634}
long_replace = {-116.2970014:139.7601, -80.8328748:139.8061, 175.7552111:139.7279, 21.424098:139.8731}

lat_new = [lat_replace.get(lat,lat) for lat in Lat]
long_new = [long_replace.get(long, long) for long in Long]


In [25]:
df['Dist_Latitude'] = lat_new
df['Dist_Longitude'] = long_new

In [26]:
Tokyo= df.drop(['Latitude', 'Longitude'], axis=1)

In [28]:
Tokyo.head(3)

,Ward,Area_SqKm,Population,District,Dist_Latitude,Dist_Longitude
1,Chiyoda,5100,59441,Nagatacho,35.675618,139.743469
2,Chuo,14460,147620,Nihonbashi,35.684068,139.774503
3,Minato,12180,248071,Odaiba,35.626722,139.772101


In [27]:
# Keeping a copy.
Tokyo.to_csv("Tokyo_Scaped.csv")

In [29]:
# Fetching Tokyo's Latitude and Longitude. 
address = 'Tokyo'
geolocator = Nominatim(user_agent="Tokyo_explorer")
location = geolocator.geocode(address)
Tokyo_latitude = location.latitude
Tokyo_longitude = location.longitude
print('The geograpical coordinates of Tokyo are {}, {}.'.format(Tokyo_latitude, Tokyo_longitude))

The geograpical coordinates of Tokyo are 35.6828387, 139.7594549.


In [34]:
Tokyo_map = folium.Map(location=[Tokyo_latitude, Tokyo_longitude], zoom_start=12)

# Adding markers.
for lat, lng, label in zip(Tokyo['Dist_Latitude'], Tokyo['Dist_Longitude'], 
                           Tokyo['District']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=9,
        popup=label,
        color='RED',
        fill=True,
        fill_color='#FFFFFF',
        fill_opacity=1).add_to(Tokyo_map)  
    
Tokyo_map

In [35]:
CLIENT_ID = 'xxxxxxxxxx' # Use your client ID
CLIENT_SECRET = 'xxxxxxxx' # Use your client secret
VERSION = '20201206' 

In [36]:
radius=550
categoryId='4bf58dd8d48988d10f941735'# This is the category Id for Indian Restaurants.
limit=50 # This is the maximum limit for search queries.

def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius,
            limit,
            categoryId)
            
        results = requests.get(url).json()['response']['venues']
        
        # Filtering relevant information.
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'Dist_Latitude', 
                  'Dist_Longitude', 
                  'Venue']
    
    return(nearby_venues)

In [37]:
Indian_restaurants = getNearbyVenues(names=df['District'],
                                   latitudes=df['Dist_Latitude'],
                                   longitudes=df['Dist_Longitude']
                                  )

Nagatacho
Nihonbashi
Odaiba
Shinjuku
Hongo
Ueno
Kinshicho
Kiba
Shinagawa
Meguro
Omori
Shimokitazawa
Shibuya
Nakano
Koenji
Ikebukuro
Akabane
Arakawa
Itabashi
Nerima
Ayase
Tateishi
Kasai


In [40]:
Indian_restaurants.shape

(561, 4)

In [41]:
a=Indian_restaurants['District'].value_counts()

In [47]:
a['Nagatacho']

38

In [45]:
Tokyo['District']

1         Nagatacho
2        Nihonbashi
3            Odaiba
4          Shinjuku
5             Hongo
6              Ueno
7         Kinshicho
8              Kiba
9         Shinagawa
10           Meguro
11            Omori
12    Shimokitazawa
13          Shibuya
14           Nakano
15           Koenji
16        Ikebukuro
17          Akabane
18          Arakawa
19         Itabashi
20           Nerima
21            Ayase
22         Tateishi
23            Kasai
Name: District, dtype: object

In [48]:
freq=[]
for i in Tokyo['District']:
    count=a[i]
    freq.append(count)

In [48]:
Tokyo['Frequency']=freq

In [49]:
Tokyo.head()

,Ward,Area_SqKm,Population,Major_District,Dist_Latitude,Dist_Longitude,Frequency
1,Chiyoda,5100,59441,Nagatacho,35.675618,139.743469,38
2,Chuo,14460,147620,Nihonbashi,35.684058,139.774501,50
3,Minato,12180,248071,Odaiba,35.619128,139.779403,2
4,Shinjuku,18620,339211,Shinjuku,35.693763,139.703632,49
5,Bunkyō,19790,223389,Hongo,35.708800,139.760100,49


In [51]:
group=[]
for freq in Tokyo["Frequency"]:
    if freq>33:
        a=0
    elif freq>17:
        a=1
    else:
        a=2
    group.append(a)

In [52]:
Tokyo['Group']=group
Tokyo.head(3)

,Ward,Area_SqKm,Population,Major_District,Dist_Latitude,Dist_Longitude,Frequency,Group
1,Chiyoda,5100,59441,Nagatacho,35.675618,139.743469,38,0
2,Chuo,14460,147620,Nihonbashi,35.684058,139.774501,50,0
3,Minato,12180,248071,Odaiba,35.619128,139.779403,2,2


In [55]:
# Creating map.
map_clusters = folium.Map(location=[35.6828387, 139.7594549], zoom_start=11)

# Setting color scheme for the clusters.
x = np.arange(3)
ys = [i + x + (i*x)**2 for i in range(3)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Adding markers to the map.
markers_colors = []
for lat, lon, poi, cluster in zip(Tokyo['Dist_Latitude'],Tokyo['Dist_Longitude'],Tokyo['Ward'],Tokyo['Group']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Note: Observations are made in the final report.

# Notebook's end.